In [1]:
import torch
import glob
import os
# glob 결과 숫자 오름차순으로 정리해주는 라이브러리, 기능적으로 필요하지 않았음을 깨달았으나
# 정렬 작업이 유지보수를 가정했을 때 충분히 의미 있다고 생각해서 그냥 놔두기로 함
import natsort
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
from torchvision import models
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import time

In [2]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
import drive.MyDrive.Colab_Notebooks.resnet_datanmodel as datanmodel

Mounted at /content/drive


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [4]:
# 경로 설정, py파일로 변환시 경로는 변경되어야 한다.
# local path
# path=os.path.abspath('../')
# colab path
path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/')

# Resize: 크기를 224, 224로 맞춘다
# ToTensor: 데이터 타입을 Tensor로 만든다. Tensor의 원소는 0~1로 정해진다.(https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html#torchvision.transforms.ToTensor)
# custom으로 transform를 작성하는 것도 가능하다.
transforms=transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()])

In [5]:
cnd_train=datanmodel.cnd_data(file_path=path, train=True, transforms=transforms)

In [6]:
batch=32
cnd_dataloader=DataLoader(cnd_train, batch_size=batch, shuffle=True)

In [7]:
# resnet34나 50이나 조금씩 원문에서 주장하는 model을 수정한 듯한 흔적이 보인다.
# 하지만 지금은 resnet 원문의 것을 구현하는 입장이기 때문에
# 모델 참조를 하다 원문과 다른 부분이 있다면 무시하고 원문대로 한다.
# resnet34_preset=models.resnet34()
# summary(resnet34_preset, input_size=(3, 224, 224))
# print(resnet34_preset)

In [8]:
# resnet50_preset=models.resnet50()
# summary(resnet50_preset, input_size=(3, 224, 224))
# print(resnet50_preset)

In [9]:
train_model=datanmodel.ResNet_compat().to(device)
# summary(train_model, input_size=(3, 224, 224))
# print(train_model)

In [10]:
learning_rate=0.01

loss_f= nn.CrossEntropyLoss()
# train_model.parameters: 최적화할 대상의 파라미터
# lr=learning_rate
optimizer = torch.optim.SGD(train_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)


In [11]:
log_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_log/')
weight_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_pth/')

In [12]:
writer=SummaryWriter(log_save_path)

In [13]:
# 폴더에 가중치가 있으면 불러온다.
# 아래와 같이 코드가 복잡한 이유는 GPU일 때와 CPU일 때의 차이가 있기 때문에 이를 맞춰줘야 했기 때문
# 또한 병렬 처리가 된 GPU의 경우 추가로 key를 생성하는데, 비병렬 상황에서는 이를 제거해야 된다.
weight_list=natsort.natsorted(glob.glob(weight_save_path+'/*.pth'), reverse=True)

if weight_list:
  start_epoch=int(weight_list[0].split('_')[-1].split('.')[0])+1
  print(f'{start_epoch+1} epoch 부터 시작합니다.')

  # GPU 사용 불가시
  if device=='cpu':
    loaded_weight=torch.load(weight_list[0], map_location=torch.device('cpu'))
    if isinstance(train_model,nn.DataParallel):
      print('cpu 병렬')
    else:
      print('cpu 병렬 x')

  # GPU 사용 가능시
  else:
    loaded_weight=torch.load(weight_list[0])
    if isinstance(train_model,nn.DataParallel):
      print('gpu 병렬')
    else:
      print('gpu 병렬 x')

  model_key=train_model.state_dict().keys()
  weight_key=loaded_weight.keys()

  diff_list=list()
  for key in weight_key:
    if key not in model_key:
      diff_list.append(key)

  for diff_key in diff_list:
    del loaded_weight[diff_key]

  train_model.load_state_dict(loaded_weight)

else:
  start_epoch=0
  print('처음부터 시작합니다.')

40 epoch 부터 시작합니다.
gpu 병렬 x


In [ ]:
# dogs 11285, 8730, 11675 3588, 5604(not dog), 11853, 2877, 6318, 9078(channel 4), 11410 /3588와 5604가 중복해서 나옴. 특정 데이터 문제일 가능성이 높아짐
# cats 8470, 5686, 9778, 2877, 7276, 11935, 5370
# 위 문제는 비트 수준(bit-depth)문제로 발생한 것이며, 이미 해결함

EPOCHS=60

for epoch in range(start_epoch, EPOCHS):
    # running loss: 5batch동안 loss 누적값
    running_loss=0
    running_acc=0

    # epoch_total_loss: 1 epoch에서 발생한 loss 누적값, 750(1 epoch batch수)을 나눠서 평균 loss값을 구하는데 사용할 예정
    epoch_total_loss=0
    epoch_total_acc=0
    start_time=time.time()


    for i, inp in enumerate(cnd_dataloader):

        input, label= inp
        input, label= input.to(device), torch.Tensor(label).to(device)
        # 모든 gradient를 0으로 설정, 이렇게 하지 않으면 이전 loop의 gradient값이 그대로 남아있어 제대로 학습이 되지 않는다.
        optimizer.zero_grad()

        # train_model을 태운 다음 loss를 계산한다.
        output= train_model(input)
        loss= loss_f(output, label)

        # accuracy를 계산한다.
        correct=0
        for t in zip(output.tolist(), label.tolist()):
          if t[0][0] >= 0.5:
              ans=0
          else:
              ans=1
          if ans==t[1]:
            correct+=1
          else:
            pass

        # loss.backward()로 gradient를 계산하고
        # optimizer를 사용하여 반영한다.
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
        running_acc+=correct

        if i%5 == 4 and i>0:
            end_time=time.time()

            print(f'Loss [{epoch+1}, {i+1}](epoch, minibatch): ', running_loss/100)
            print(f'Accuracy [{epoch+1}, {i+1}](epoch, minibatch): ', running_acc/(batch*5))
            print('time taken(per 5 batch):', end_time-start_time)
            start_time=end_time
            epoch_total_loss+=running_loss
            epoch_total_acc+=running_acc
            running_loss=0.0
            running_acc=0

    torch.save(train_model.state_dict(), os.path.join(weight_save_path, f'model_weights_{epoch}.pth'))
    writer.add_scalar("Loss / epoch ", epoch_total_loss/len(cnd_train), epoch)
    writer.add_scalar("Accuracy / epoch", epoch_total_acc/len(cnd_train), epoch)


writer.close()


Loss [40, 5](epoch, minibatch):  0.016976864635944368
Accuracy [40, 5](epoch, minibatch):  0.975
time taken(per 5 batch): 135.1524269580841
Loss [40, 10](epoch, minibatch):  0.017187028527259826
Accuracy [40, 10](epoch, minibatch):  0.96875
time taken(per 5 batch): 126.47040438652039
Loss [40, 15](epoch, minibatch):  0.01630682796239853
Accuracy [40, 15](epoch, minibatch):  0.9875
time taken(per 5 batch): 127.3164496421814
Loss [40, 20](epoch, minibatch):  0.018357443511486052
Accuracy [40, 20](epoch, minibatch):  0.94375
time taken(per 5 batch): 126.85471320152283
Loss [40, 25](epoch, minibatch):  0.016971471905708312
Accuracy [40, 25](epoch, minibatch):  0.975
time taken(per 5 batch): 123.07344603538513
Loss [40, 30](epoch, minibatch):  0.01633536398410797
Accuracy [40, 30](epoch, minibatch):  0.9875
time taken(per 5 batch): 127.36613249778748
Loss [40, 35](epoch, minibatch):  0.01701154887676239
Accuracy [40, 35](epoch, minibatch):  0.975
time taken(per 5 batch): 127.37074947357178


아래는 가중치 확인

In [ ]:
raise Exception("에러 메시지")

In [ ]:
# GPU 사용 불가시
if device=='cpu':
  if isinstance(train_model,nn.DataParallel):
    print('cpu 병렬')
  else:

    print('cpu 병렬 x')

# GPU 사용 가능시
else:
  if isinstance(train_model,nn.DataParallel):
    print('gpu 병렬')
  else:
    print('gpu 병렬 x')

In [ ]:
weight_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_pth/')
weight_list=natsort.natsorted(glob.glob(weight_save_path+'/*.pth'), reverse=True)
loaded_weight=torch.load(weight_list[0], map_location=torch.device('cpu'))

In [ ]:
model=train_model.state_dict().keys()
weight=loaded_weight.keys()

# model_keys=set(train_model.state_dict().keys())
# weight_keys=set(loaded_weight.keys())

for key in weight:
  if key not in model:
    print(key)

In [ ]:
type(loaded_weight.keys())

In [ ]:
len(train_model.state_dict().keys())

In [ ]:
if 'reduce.weight' in loaded_weight.keys():

In [ ]:
del loaded_weight['reduce.bias']

In [ ]:
del loaded_weight['reduce.weight']

In [ ]:
train_model.load_state_dict(loaded_weight)